# Wide & Deep Recommender System

#### Students Group Number: 1
#### Students Name and ID:

## 1. Imports and Configuration

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import gc
import os
import random
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

In [4]:
# === CONFIGURATION ===
@dataclass
class Config:
    # Embeddings
    embedding_dim: int = 32
    embedding_dropout: float = 0.25  # Moderate dropout
    
    # Deep tower - Shallower [256, 128]
    deep_layers: Tuple[int, ...] = (256, 128)  # Removed 3rd layer
    deep_dropout: float = 0.35  # Moderate dropout
    
    # Wide tower
    wide_hidden_dim: int = 32
    wide_dropout: float = 0.35  # Moderate dropout
    
    # Biases
    bias_dropout: float = 0.1
    
    # Optimizer - Moderate weight decay
    lr: float = 5e-4
    weight_decay: float = 5e-3  # Moderate weight decay (0.005)
    
    # Scheduler
    warmup_epochs: int = 2
    scheduler_factor: float = 0.5
    scheduler_patience: int = 3
    min_lr: float = 1e-5
    
    # Training
    n_epochs: int = 25
    batch_size: int = 2048
    patience: int = 6
    grad_clip: float = 0.9
    
    # Features - Standard smoothing (no adaptive)
    smoothing_strength: float = 80.0  # Standard Bayesian smoothing
    max_tags: int = 80
    
    # Bucketing thresholds
    user_activity_thresholds: Tuple[int, ...] = (10, 40, 100)
    item_popularity_thresholds: Tuple[int, ...] = (30, 150, 400)
    
    # Ensemble
    seeds: Tuple[int, ...] = (42, 123, 456, 789, 2025)
    n_seeds_to_use: int = 5
    
    # Larger validation set for better estimate
    val_ratio: float = 0.25  # 25% validation
    final_val_ratio: float = 0.05  # For retrain phase


CONFIG = Config()
print("Configuration loaded.")

Configuration loaded.


In [5]:
# Device configuration
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("✓ Using MPS (Apple Silicon GPU)")
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("✓ Using CUDA GPU")
else:
    DEVICE = torch.device("cpu")
    print("✓ Using CPU")

print(f"PyTorch: {torch.__version__}")

✓ Using MPS (Apple Silicon GPU)
PyTorch: 2.9.1


In [6]:
# === DATA PATHS ===
DATA_DIR = "../project/recsys-runi-2026"
if not os.path.exists(DATA_DIR):
    DATA_DIR = "recsys-runi-2026"
if not os.path.exists(DATA_DIR):
    DATA_DIR = "."
print(f"Data directory: {DATA_DIR}")

Data directory: recsys-runi-2026


## 2. Utility Functions

In [7]:
def set_all_seeds(seed: int):
    """Set seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [8]:
def normalize_tag(tag: str) -> str:
    if pd.isna(tag):
        return ""
    tag = str(tag).lower()
    tag = re.sub(r'[^a-z0-9\s]', '', tag)
    tag = re.sub(r'\s+', ' ', tag)
    return tag.strip()


def extract_movie_year(title: str) -> Tuple[str, Optional[int]]:
    if pd.isna(title):
        return "", None
    match = re.search(r'\((\d{4})(?:-\d{4})?\)\s*$', title)
    if match:
        return title.strip(), int(match.group(1))
    return title.strip(), None

## 3. Data Splitting Functions

In [9]:
def user_coldstart_split(df: pd.DataFrame, val_user_ratio: float = 0.15, random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split by USERS - hold out val_user_ratio% of users entirely.
    This prevents leakage by ensuring validation users are completely unseen during training.
    Simulates cold-start scenario which better matches test distribution.
    """
    print(f"\n[USER COLD-START SPLIT] Holding out {val_user_ratio*100:.0f}% of users...")
    
    np.random.seed(random_state)
    
    # Get all unique users
    all_users = df['user_id'].unique()
    n_users = len(all_users)
    n_val_users = int(n_users * val_user_ratio)
    
    # Shuffle and split users
    np.random.shuffle(all_users)
    val_users = set(all_users[:n_val_users])
    train_users = set(all_users[n_val_users:])
    
    # Split dataframe by users
    train_df = df[df['user_id'].isin(train_users)].reset_index(drop=True)
    val_df = df[df['user_id'].isin(val_users)].reset_index(drop=True)
    
    print(f"  Train users: {len(train_users):,}, Train ratings: {len(train_df):,}")
    print(f"  Val users: {len(val_users):,}, Val ratings: {len(val_df):,}")
    print(f"  Avg ratings per train user: {len(train_df)/len(train_users):.1f}")
    print(f"  Avg ratings per val user: {len(val_df)/len(val_users):.1f}")
    
    return train_df, val_df


def item_coldstart_split(df: pd.DataFrame, val_item_ratio: float = 0.10, random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split by ITEMS - hold out val_item_ratio% of movies entirely.
    This simulates cold-start scenario for new items not seen during training.
    """
    print(f"\n[ITEM COLD-START SPLIT] Holding out {val_item_ratio*100:.0f}% of items...")
    
    np.random.seed(random_state)
    
    # Get all unique items
    all_items = df['movie_id'].unique()
    n_items = len(all_items)
    n_val_items = int(n_items * val_item_ratio)
    
    # Shuffle and split items
    np.random.shuffle(all_items)
    val_items = set(all_items[:n_val_items])
    train_items = set(all_items[n_val_items:])
    
    # Split dataframe by items
    train_df = df[df['movie_id'].isin(train_items)].reset_index(drop=True)
    val_df = df[df['movie_id'].isin(val_items)].reset_index(drop=True)
    
    print(f"  Train items: {len(train_items):,}, Train ratings: {len(train_df):,}")
    print(f"  Val items (cold): {len(val_items):,}, Val ratings: {len(val_df):,}")
    print(f"  Avg ratings per train item: {len(train_df)/len(train_items):.1f}")
    print(f"  Avg ratings per val item: {len(val_df)/len(val_items):.1f}")
    
    return train_df, val_df


def stratified_rating_split(df: pd.DataFrame, val_ratio: float = 0.05, random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Stratified split for retrain phase - take val_ratio of each user's ratings.
    """
    np.random.seed(random_state)
    train_indices, val_indices = [], []
    
    for user_id, group in df.groupby('user_id'):
        indices = group.index.tolist()
        np.random.shuffle(indices)
        n_val = max(1, int(len(indices) * val_ratio))
        train_indices.extend(indices[n_val:])
        val_indices.extend(indices[:n_val])
    
    return df.loc[train_indices].reset_index(drop=True), df.loc[val_indices].reset_index(drop=True)

## 4. Feature Store

In [10]:
class FeatureStore:
    """
    Simplified feature store - no adaptive smoothing, no complex stats.
    Target encoding computed ONLY on training data to prevent leakage.
    """
    def __init__(self):
        self.user_id_to_idx = {}
        self.item_id_to_idx = {}
        self.n_users = 0
        self.n_items = 0
        self.genre_list = []
        self.genre_features = {}
        self.movie_years = {}
        self.movie_year_bucket = {}
        
        # Standard target encoding (computed on train only)
        self.user_mean_rating = {}
        self.item_mean_rating = {}
        self.user_rating_count = {}
        self.item_rating_count = {}
        self.user_activity_bucket = {}
        self.item_popularity_bucket = {}
        
        self.global_mean = 3.5
    
    def build_basic(self, train_df, submission_df, movies_df):
        """Build ID mappings and movie metadata."""
        print("\n" + "="*60)
        print("BUILDING FEATURE STORE (Temporal Generalization)")
        print("="*60)
        
        # ID mappings (include all users/items from train + submission)
        print("[1/3] Building ID mappings...")
        all_users = set(train_df['user_id'].unique()) | set(submission_df['user_id'].unique())
        all_items = set(train_df['movie_id'].unique()) | set(submission_df['movie_id'].unique()) | set(movies_df['movie_id'].unique())
        self.user_id_to_idx = {uid: idx for idx, uid in enumerate(sorted(all_users))}
        self.item_id_to_idx = {iid: idx for idx, iid in enumerate(sorted(all_items))}
        self.n_users = len(self.user_id_to_idx)
        self.n_items = len(self.item_id_to_idx)
        print(f"  Users: {self.n_users:,}, Items: {self.n_items:,}")
        
        # Movie metadata
        print("[2/3] Extracting movie metadata...")
        for _, row in movies_df.iterrows():
            mid = row['movie_id']
            _, year = extract_movie_year(row['title'])
            self.movie_years[mid] = year
            if year:
                if year < 1970: self.movie_year_bucket[mid] = 0
                elif year >= 2010: self.movie_year_bucket[mid] = 5
                else: self.movie_year_bucket[mid] = min(5, (year - 1970) // 10 + 1)
            else:
                self.movie_year_bucket[mid] = 3
        
        # Genre features
        print("[3/3] Building genre features...")
        all_genres = set()
        for g in movies_df['genres'].dropna():
            if g != '(no genres listed)':
                all_genres.update(g.split('|'))
        self.genre_list = sorted(list(all_genres))
        
        for _, row in movies_df.iterrows():
            mid = row['movie_id']
            genres = row['genres'].split('|') if pd.notna(row['genres']) and row['genres'] != '(no genres listed)' else []
            self.genre_features[mid] = np.array([1.0 if g in genres else 0.0 for g in self.genre_list], dtype=np.float32)
        print(f"  Genres: {len(self.genre_list)}")
        print("="*60)
    
    def build_target_encoding(self, train_df: pd.DataFrame):
        """
        Standard Bayesian target encoding - computed ONLY on training data.
        No adaptive smoothing - fixed smoothing strength.
        """
        print(f"\nBuilding TARGET ENCODING (train data only)...")
        print(f"  Smoothing strength: {CONFIG.smoothing_strength}")
        
        explicit = train_df[train_df['rating'].notna()]
        self.global_mean = explicit['rating'].mean()
        
        # User stats with standard smoothing
        user_stats = explicit.groupby('user_id')['rating'].agg(['mean', 'count'])
        for uid, row in user_stats.iterrows():
            n, avg = row['count'], row['mean']
            # Standard Bayesian smoothing: (n * avg + m * global_mean) / (n + m)
            smoothed = (n * avg + CONFIG.smoothing_strength * self.global_mean) / (n + CONFIG.smoothing_strength)
            self.user_mean_rating[uid] = smoothed
            self.user_rating_count[uid] = n
            
            thresholds = CONFIG.user_activity_thresholds
            if n < thresholds[0]: self.user_activity_bucket[uid] = 0
            elif n < thresholds[1]: self.user_activity_bucket[uid] = 1
            elif n < thresholds[2]: self.user_activity_bucket[uid] = 2
            else: self.user_activity_bucket[uid] = 3
        
        # Item stats with standard smoothing
        item_stats = explicit.groupby('movie_id')['rating'].agg(['mean', 'count'])
        for mid, row in item_stats.iterrows():
            n, avg = row['count'], row['mean']
            smoothed = (n * avg + CONFIG.smoothing_strength * self.global_mean) / (n + CONFIG.smoothing_strength)
            self.item_mean_rating[mid] = smoothed
            self.item_rating_count[mid] = n
            
            thresholds = CONFIG.item_popularity_thresholds
            if n < thresholds[0]: self.item_popularity_bucket[mid] = 0
            elif n < thresholds[1]: self.item_popularity_bucket[mid] = 1
            elif n < thresholds[2]: self.item_popularity_bucket[mid] = 2
            else: self.item_popularity_bucket[mid] = 3
        
        print(f"  Global mean: {self.global_mean:.4f}")
        print(f"  Users with ratings: {len(self.user_mean_rating):,}")
        print(f"  Items with ratings: {len(self.item_mean_rating):,}")


FEATURES = FeatureStore()
print("Feature store initialized.")

Feature store initialized.


## 5. Wide & Deep Model Architecture

In [11]:
class WideDeepModel(nn.Module):
    """
    Simplified architecture for better generalization.
    - Shallower deep tower: [256, 128]
    - Consistent high dropout: 0.4
    - Removed complex continuous features
    """
    
    def __init__(self, n_users, n_items, n_genres, global_mean=3.5):
        super().__init__()
        
        self.global_mean = nn.Parameter(torch.tensor([global_mean]), requires_grad=False)
        
        # === BIASES ===
        self.user_bias = nn.Embedding(n_users, 1)
        self.item_bias = nn.Embedding(n_items, 1)
        self.bias_dropout = nn.Dropout(CONFIG.bias_dropout)
        
        # === EMBEDDINGS ===
        self.user_emb = nn.Embedding(n_users, CONFIG.embedding_dim)
        self.item_emb = nn.Embedding(n_items, CONFIG.embedding_dim)
        self.emb_dropout = nn.Dropout(CONFIG.embedding_dropout)
        
        # === WIDE PART ===
        # Features: genres(19) + year_bucket(6) + user_activity(4) + item_pop(4) = 33
        wide_input_dim = n_genres + 6 + 4 + 4
        self.wide_hidden = nn.Linear(wide_input_dim, CONFIG.wide_hidden_dim)
        self.wide_bn = nn.BatchNorm1d(CONFIG.wide_hidden_dim)
        self.wide_dropout = nn.Dropout(CONFIG.wide_dropout)
        self.wide_output = nn.Linear(CONFIG.wide_hidden_dim, 1)
        
        # === DEEP PART (Simplified) ===
        # Input: user_emb(32) + item_emb(32) + genres(19) + year_normalized(1) = 84
        deep_input_dim = CONFIG.embedding_dim * 2 + n_genres + 1
        
        self.deep_layers = nn.ModuleList()
        self.deep_bns = nn.ModuleList()
        self.deep_dropouts = nn.ModuleList()
        
        prev_dim = deep_input_dim
        for hidden_dim in CONFIG.deep_layers:
            self.deep_layers.append(nn.Linear(prev_dim, hidden_dim))
            self.deep_bns.append(nn.BatchNorm1d(hidden_dim))
            self.deep_dropouts.append(nn.Dropout(CONFIG.deep_dropout))
            prev_dim = hidden_dim
        
        self.deep_output = nn.Linear(CONFIG.deep_layers[-1], 1)
        self._init_weights()
    
    def _init_weights(self):
        nn.init.zeros_(self.user_bias.weight)
        nn.init.zeros_(self.item_bias.weight)
        nn.init.normal_(self.user_emb.weight, 0, 0.01)
        nn.init.normal_(self.item_emb.weight, 0, 0.01)
        
        nn.init.xavier_uniform_(self.wide_hidden.weight)
        nn.init.zeros_(self.wide_hidden.bias)
        nn.init.xavier_uniform_(self.wide_output.weight)
        nn.init.zeros_(self.wide_output.bias)
        
        for layer in self.deep_layers:
            nn.init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
            nn.init.zeros_(layer.bias)
        nn.init.xavier_uniform_(self.deep_output.weight)
        nn.init.zeros_(self.deep_output.bias)
    
    def forward(self, user_idx, item_idx, genre, wide_features, year_normalized):
        # === BIASES ===
        u_bias = self.user_bias(user_idx).squeeze(-1)
        i_bias = self.item_bias(item_idx).squeeze(-1)
        if self.training:
            u_bias = self.bias_dropout(u_bias.unsqueeze(-1)).squeeze(-1) / (1 - CONFIG.bias_dropout)
            i_bias = self.bias_dropout(i_bias.unsqueeze(-1)).squeeze(-1) / (1 - CONFIG.bias_dropout)
        
        # === WIDE ===
        wide_h = self.wide_dropout(F.relu(self.wide_bn(self.wide_hidden(wide_features))))
        wide_out = self.wide_output(wide_h).squeeze(-1)
        
        # === EMBEDDINGS ===
        u_emb = self.emb_dropout(self.user_emb(user_idx))
        i_emb = self.emb_dropout(self.item_emb(item_idx))
        
        # === DEEP ===
        deep_in = torch.cat([u_emb, i_emb, genre, year_normalized], dim=1)
        
        x = deep_in
        for layer, bn, dropout in zip(self.deep_layers, self.deep_bns, self.deep_dropouts):
            x = dropout(F.relu(bn(layer(x))))
        
        deep_out = self.deep_output(x).squeeze(-1)
        
        # === FINAL ===
        return self.global_mean + u_bias + i_bias + wide_out + deep_out

## 6. Feature Preparation

In [12]:
def prepare_features(user_ids, movie_ids):
    """Simplified feature preparation - no complex statistics."""
    n = len(user_ids)
    n_genres = len(FEATURES.genre_list)
    
    # Genre features
    genre = np.zeros((n, n_genres), dtype=np.float32)
    for i, mid in enumerate(movie_ids):
        if mid in FEATURES.genre_features:
            genre[i] = FEATURES.genre_features[mid]
    
    # Wide features: genres + year_bucket + user_activity + item_pop
    wide_dim = n_genres + 6 + 4 + 4
    wide_features = np.zeros((n, wide_dim), dtype=np.float32)
    
    for i, (uid, mid) in enumerate(zip(user_ids, movie_ids)):
        offset = 0
        
        # Genres
        if mid in FEATURES.genre_features:
            wide_features[i, :n_genres] = FEATURES.genre_features[mid]
        offset += n_genres
        
        # Year bucket (one-hot, 6 buckets)
        year_bucket = FEATURES.movie_year_bucket.get(mid, 3)
        wide_features[i, offset + year_bucket] = 1.0
        offset += 6
        
        # User activity bucket (one-hot, 4 buckets)
        activity = FEATURES.user_activity_bucket.get(uid, 1)
        wide_features[i, offset + activity] = 1.0
        offset += 4
        
        # Item popularity bucket (one-hot, 4 buckets)
        pop = FEATURES.item_popularity_bucket.get(mid, 1)
        wide_features[i, offset + pop] = 1.0
    
    # Year normalized (single continuous feature for deep tower)
    year_normalized = np.zeros((n, 1), dtype=np.float32)
    for i, mid in enumerate(movie_ids):
        year = FEATURES.movie_years.get(mid)
        if year:
            year_normalized[i, 0] = (year - 1990) / 30.0  # Normalize to roughly [-1, 1]
    
    return (torch.from_numpy(genre),
            torch.from_numpy(wide_features),
            torch.from_numpy(year_normalized))

## 7. Training Function

In [13]:
def train_model(train_df, val_df, seed=42):
    """
    Training with high regularization and temporal validation.
    """
    print(f"\n{'='*60}")
    print(f"TRAINING WIDE & DEEP (seed={seed})")
    print("="*60)
    print(f"Config: emb={CONFIG.embedding_dim}, layers={CONFIG.deep_layers}")
    print(f"        dropout={CONFIG.deep_dropout}, weight_decay={CONFIG.weight_decay}")
    print(f"        lr={CONFIG.lr}, patience={CONFIG.patience}")
    
    set_all_seeds(seed)
    
    global_mean = float(train_df['rating'].mean())
    n_users, n_items = FEATURES.n_users, FEATURES.n_items
    n_genres = len(FEATURES.genre_list)
    
    # Prepare features
    print("\nPreparing features...")
    train_user_ids = train_df['user_id'].values
    train_movie_ids = train_df['movie_id'].values
    train_user_idx = np.array([FEATURES.user_id_to_idx.get(u, 0) for u in train_user_ids], dtype=np.int64)
    train_item_idx = np.array([FEATURES.item_id_to_idx.get(m, 0) for m in train_movie_ids], dtype=np.int64)
    train_ratings = train_df['rating'].values.astype(np.float32)
    train_genre, train_wide, train_year = prepare_features(train_user_ids, train_movie_ids)
    
    val_user_ids = val_df['user_id'].values
    val_movie_ids = val_df['movie_id'].values
    val_user_idx = np.array([FEATURES.user_id_to_idx.get(u, 0) for u in val_user_ids], dtype=np.int64)
    val_item_idx = np.array([FEATURES.item_id_to_idx.get(m, 0) for m in val_movie_ids], dtype=np.int64)
    val_ratings = val_df['rating'].values.astype(np.float32)
    val_genre, val_wide, val_year = prepare_features(val_user_ids, val_movie_ids)
    
    print(f"Train: {len(train_ratings):,}, Val: {len(val_ratings):,}")
    
    # Model
    model = WideDeepModel(n_users, n_items, n_genres, global_mean=global_mean).to(DEVICE)
    
    n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Model parameters: {n_params:,}")
    
    # Optimizer with high weight decay
    optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG.lr, weight_decay=CONFIG.weight_decay)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=CONFIG.scheduler_factor, 
        patience=CONFIG.scheduler_patience, min_lr=CONFIG.min_lr
    )
    
    # Move to device
    train_user_t = torch.from_numpy(train_user_idx).to(DEVICE)
    train_item_t = torch.from_numpy(train_item_idx).to(DEVICE)
    train_rating_t = torch.from_numpy(train_ratings).to(DEVICE)
    train_genre = train_genre.to(DEVICE)
    train_wide = train_wide.to(DEVICE)
    train_year = train_year.to(DEVICE)
    
    val_user_t = torch.from_numpy(val_user_idx).to(DEVICE)
    val_item_t = torch.from_numpy(val_item_idx).to(DEVICE)
    val_rating_t = torch.from_numpy(val_ratings).to(DEVICE)
    val_genre = val_genre.to(DEVICE)
    val_wide = val_wide.to(DEVICE)
    val_year = val_year.to(DEVICE)
    
    # Training state
    best_val_rmse = float('inf')
    patience_cnt = 0
    best_state = None
    best_epoch = 0
    
    n_train = len(train_ratings)
    n_batches = (n_train + CONFIG.batch_size - 1) // CONFIG.batch_size
    
    print("\nTraining...")
    for epoch in range(CONFIG.n_epochs):
        model.train()
        
        # Learning rate warmup
        if epoch < CONFIG.warmup_epochs:
            warmup_factor = (epoch + 1) / CONFIG.warmup_epochs
            for pg in optimizer.param_groups:
                pg['lr'] = CONFIG.lr * warmup_factor
        
        perm = torch.randperm(n_train, device=DEVICE)
        epoch_loss = 0.0
        
        for b in range(n_batches):
            s, e = b * CONFIG.batch_size, min((b + 1) * CONFIG.batch_size, n_train)
            idx = perm[s:e]
            
            pred = model(train_user_t[idx], train_item_t[idx],
                        train_genre[idx], train_wide[idx], train_year[idx])
            loss = F.mse_loss(pred, train_rating_t[idx])
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=CONFIG.grad_clip)
            optimizer.step()
            epoch_loss += loss.item() * (e - s)
        
        train_rmse = np.sqrt(epoch_loss / n_train)
        
        model.eval()
        with torch.no_grad():
            val_pred = model(val_user_t, val_item_t, val_genre, val_wide, val_year)
            val_rmse = np.sqrt(F.mse_loss(val_pred, val_rating_t).item())
        
        # Step scheduler after warmup
        if epoch >= CONFIG.warmup_epochs:
            scheduler.step(val_rmse)
        
        gap = train_rmse - val_rmse
        status = "OK" if gap > -0.05 else "WARNING" if gap > -0.1 else "OVERFIT!"
        current_lr = optimizer.param_groups[0]['lr']
        
        print(f"  Epoch {epoch+1:2d}: Train={train_rmse:.4f}, Val={val_rmse:.4f}, Gap={gap:+.4f} [{status}], LR={current_lr:.6f}")
        
        # Early stopping
        if val_rmse < best_val_rmse:
            best_val_rmse = val_rmse
            best_epoch = epoch + 1
            patience_cnt = 0
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        else:
            patience_cnt += 1
            if patience_cnt >= CONFIG.patience:
                print(f"  Early stopping at epoch {epoch+1}")
                break
        
        if DEVICE.type == 'mps': torch.mps.empty_cache()
    
    if best_state:
        model.load_state_dict({k: v.to(DEVICE) for k, v in best_state.items()})
    
    print(f"\n✓ Best Epoch: {best_epoch}, Val RMSE: {best_val_rmse:.4f}")
    gc.collect()
    return model, best_val_rmse, best_epoch

## 8. Prediction Function

In [14]:
def predict_batch(model, user_ids, movie_ids, batch_size=8192):
    model.eval()
    all_preds = []
    
    for start in range(0, len(user_ids), batch_size):
        end = min(start + batch_size, len(user_ids))
        batch_users, batch_movies = user_ids[start:end], movie_ids[start:end]
        
        user_idx = np.array([FEATURES.user_id_to_idx.get(u, 0) for u in batch_users], dtype=np.int64)
        item_idx = np.array([FEATURES.item_id_to_idx.get(m, 0) for m in batch_movies], dtype=np.int64)
        genre, wide, year = prepare_features(batch_users, batch_movies)
        
        with torch.no_grad():
            preds = model(torch.from_numpy(user_idx).to(DEVICE),
                         torch.from_numpy(item_idx).to(DEVICE),
                         genre.to(DEVICE), wide.to(DEVICE), year.to(DEVICE))
        all_preds.append(preds.cpu().numpy())
        if DEVICE.type == 'mps': torch.mps.empty_cache()
    
    return np.concatenate(all_preds)

## 9. Main Training Pipeline

In [15]:
print("\n" + "="*70)
print("WIDE & DEEP - SIMPLIFIED ARCHITECTURE")
print("="*70)

# Load data
print("\nLoading data...")
train_df = pd.read_csv(f"{DATA_DIR}/train.csv")
movies_df = pd.read_csv(f"{DATA_DIR}/movies.csv")
submission_df = pd.read_csv(f"{DATA_DIR}/ratings_submission.csv")

split_ids = submission_df['id'].str.split('_', expand=True)
submission_df['user_id'] = split_ids[0].astype('int32')
submission_df['movie_id'] = split_ids[1].astype('int32')

train_explicit = train_df[train_df['rating'].notna()].copy()
print(f"Explicit ratings: {len(train_explicit):,}")


WIDE & DEEP - SIMPLIFIED ARCHITECTURE

Loading data...
Explicit ratings: 7,303,350


In [16]:
# Build basic features (ID mappings, genres, years)
FEATURES.build_basic(train_df, submission_df, movies_df)
gc.collect()


BUILDING FEATURE STORE (Temporal Generalization)
[1/3] Building ID mappings...
  Users: 100,000, Items: 2,000
[2/3] Extracting movie metadata...
[3/3] Building genre features...
  Genres: 19


0

In [17]:
# === STRATIFIED USER SPLIT (25% val) ===
print("\n" + "="*60)
print("STRATIFIED USER SPLIT (25% validation)")
print("  Larger validation for better test RMSE estimate")
print("="*60)

train_split, val_split = stratified_rating_split(train_explicit, val_ratio=CONFIG.val_ratio)
print(f"  Train: {len(train_split):,} ratings")
print(f"  Val: {len(val_split):,} ratings")

# Build target encoding on full data (users appear in both train/val)
FEATURES.build_target_encoding(train_explicit)


STRATIFIED USER SPLIT (25% validation)
  Larger validation for better test RMSE estimate
  Train: 5,514,044 ratings
  Val: 1,789,306 ratings

Building TARGET ENCODING (train data only)...
  Smoothing strength: 80.0
  Global mean: 3.6077
  Users with ratings: 100,000
  Items with ratings: 2,000


In [17]:
# === ENSEMBLE TRAINING ===
print("\n" + "="*60)
n_seeds = min(CONFIG.n_seeds_to_use, len(CONFIG.seeds))
seeds_to_use = CONFIG.seeds[:n_seeds]
print(f"TRAINING ENSEMBLE ({n_seeds} models)")
print("="*60)

models = []
val_rmses = []
best_epochs = []

for seed in seeds_to_use:
    model, val_rmse, best_epoch = train_model(train_split, val_split, seed=seed)
    models.append(model)
    val_rmses.append(val_rmse)
    best_epochs.append(best_epoch)
    gc.collect()
    if DEVICE.type == 'mps': torch.mps.empty_cache()

print(f"\n--- Per-Model Summary ---")
for i, (seed, vr, be) in enumerate(zip(seeds_to_use, val_rmses, best_epochs)):
    print(f"  Model {i+1} (seed={seed}): Val RMSE={vr:.4f}, Best Epoch={be}")
print(f"  Average Val RMSE: {np.mean(val_rmses):.4f}")
print(f"  Std Val RMSE: {np.std(val_rmses):.4f}")


TRAINING ENSEMBLE (5 models)

TRAINING WIDE & DEEP (seed=42)
Config: emb=32, layers=(256, 128)
        dropout=0.35, weight_decay=0.005
        lr=0.0005, patience=6

Preparing features...
Train: 5,514,044, Val: 1,789,306
Model parameters: 3,422,738

Training...
  Epoch  1: Train=0.9720, Val=0.8493, Gap=+0.1227 [OK], LR=0.000250
  Epoch  2: Train=0.8494, Val=0.8332, Gap=+0.0162 [OK], LR=0.000500
  Epoch  3: Train=0.8278, Val=0.8220, Gap=+0.0058 [OK], LR=0.000500
  Epoch  4: Train=0.8131, Val=0.8132, Gap=-0.0001 [OK], LR=0.000500
  Epoch  5: Train=0.7998, Val=0.8075, Gap=-0.0077 [OK], LR=0.000500
  Epoch  6: Train=0.7893, Val=0.8036, Gap=-0.0143 [OK], LR=0.000500
  Epoch  7: Train=0.7808, Val=0.8007, Gap=-0.0199 [OK], LR=0.000500
  Epoch  8: Train=0.7738, Val=0.7984, Gap=-0.0245 [OK], LR=0.000500
  Epoch  9: Train=0.7679, Val=0.7966, Gap=-0.0287 [OK], LR=0.000500
  Epoch 10: Train=0.7631, Val=0.7960, Gap=-0.0330 [OK], LR=0.000500
  Epoch 11: Train=0.7587, Val=0.7951, Gap=-0.0363 [OK], 

In [18]:
# === ENSEMBLE EVALUATION ===
print("\n" + "="*60)
print("ENSEMBLE EVALUATION (Temporal Validation)")
print("="*60)

val_preds_list = []
for model in models:
    preds = predict_batch(model, val_split['user_id'].values, val_split['movie_id'].values)
    val_preds_list.append(preds)

val_preds_array = np.array(val_preds_list)
ensemble_val_preds = val_preds_array.mean(axis=0)
val_targets = val_split['rating'].values
ensemble_rmse = np.sqrt(np.mean((ensemble_val_preds - val_targets) ** 2))

pred_std = val_preds_array.std(axis=0)

print(f"\n  ensemble_val_rmse (temporal): {ensemble_rmse:.4f}")
print(f"  prediction_std_mean: {pred_std.mean():.4f}")
print(f"  prediction_std_95th_percentile: {np.percentile(pred_std, 95):.4f}")
print(f"  val_prediction_mean: {ensemble_val_preds.mean():.4f}")


ENSEMBLE EVALUATION (Temporal Validation)

  ensemble_val_rmse (temporal): 0.7846
  prediction_std_mean: 0.0950
  prediction_std_95th_percentile: 0.2084
  val_prediction_mean: 3.6043


In [19]:
# === RETRAIN ON FULL DATA ===
print("\n" + "="*60)
print("RETRAINING ENSEMBLE ON FULL DATA")
print("="*60)

# For final training, use stratified split with smaller val ratio
final_train, final_val = stratified_rating_split(train_explicit, val_ratio=CONFIG.final_val_ratio)

# Rebuild target encoding on final training data
FEATURES.build_target_encoding(final_train)

final_models = []
for seed in seeds_to_use:
    model, _, _ = train_model(final_train, final_val, seed=seed)
    final_models.append(model)
    gc.collect()
    if DEVICE.type == 'mps': torch.mps.empty_cache()


RETRAINING ENSEMBLE ON FULL DATA

Building TARGET ENCODING (train data only)...
  Smoothing strength: 80.0
  Global mean: 3.6081
  Users with ratings: 100,000
  Items with ratings: 2,000

TRAINING WIDE & DEEP (seed=42)
Config: emb=32, layers=(256, 128)
        dropout=0.35, weight_decay=0.005
        lr=0.0005, patience=6

Preparing features...
Train: 6,980,647, Val: 322,703
Model parameters: 3,422,738

Training...
  Epoch  1: Train=0.9484, Val=0.8452, Gap=+0.1032 [OK], LR=0.000250
  Epoch  2: Train=0.8441, Val=0.8256, Gap=+0.0186 [OK], LR=0.000500
  Epoch  3: Train=0.8245, Val=0.8151, Gap=+0.0094 [OK], LR=0.000500
  Epoch  4: Train=0.8106, Val=0.8054, Gap=+0.0052 [OK], LR=0.000500
  Epoch  5: Train=0.7976, Val=0.7984, Gap=-0.0009 [OK], LR=0.000500
  Epoch  6: Train=0.7871, Val=0.7936, Gap=-0.0065 [OK], LR=0.000500
  Epoch  7: Train=0.7795, Val=0.7898, Gap=-0.0103 [OK], LR=0.000500
  Epoch  8: Train=0.7739, Val=0.7880, Gap=-0.0141 [OK], LR=0.000500
  Epoch  9: Train=0.7694, Val=0.7870

In [20]:
# === GENERATE SUBMISSION ===
print("\n" + "="*60)
print("GENERATING ENSEMBLE SUBMISSION")
print("="*60)

# Get ensemble predictions
sub_preds_list = []
for model in final_models:
    preds = predict_batch(model, submission_df['user_id'].values, submission_df['movie_id'].values)
    sub_preds_list.append(preds)

ensemble_sub_preds = np.mean(sub_preds_list, axis=0)

# Clip to valid range
ensemble_sub_preds = np.clip(ensemble_sub_preds, 0.5, 5.0)

submission = pd.DataFrame({
    'id': submission_df['id'],
    'prediction': ensemble_sub_preds
})

output_file = 'submission_wide_deep.csv'
submission.to_csv(output_file, index=False)

print(f"\n✓ Saved: {output_file}")
print(f"Predictions: {len(submission):,}")
print(f"Rating range: [{ensemble_sub_preds.min():.2f}, {ensemble_sub_preds.max():.2f}]")
print(f"Rating mean (submission): {ensemble_sub_preds.mean():.4f}")

print("\n" + "="*70)
print(f"EXPECTED TEST RMSE (25% val): ~{ensemble_rmse:.4f}")
print("  Shallower model [256,128], moderate regularization")
print("="*70)


GENERATING ENSEMBLE SUBMISSION

✓ Saved: submission_wide_deep.csv
Predictions: 100,000
Rating range: [0.53, 5.00]
Rating mean (submission): 3.7024

EXPECTED TEST RMSE (25% val): ~0.7846
  Shallower model [256,128], moderate regularization


---

# Cold Start Strategy

This section addresses how the Wide & Deep model handles the cold-start problem in movie recommender systems.

## 1. The Cold-Start Problem in Movie Recommender Systems

### What is Cold Start?

The **cold-start problem** occurs when a recommender system must make predictions for entities it has never seen during training:

- **New Users (User Cold Start)**: Users who have no or very few ratings in the training data. The system lacks collaborative signals to understand their preferences.

- **New Items (Item Cold Start)**: Movies that have never been rated or have very few ratings. The system lacks user feedback to learn the item's quality or appeal.

### Why is Cold Start Challenging?

In collaborative filtering approaches, the model learns **user embeddings** and **item embeddings**. For new users/items:
- Embeddings are random/zero-initialized → no useful information
- Predictions default to global averages
- Poor personalization and recommendation quality

## 2. Proposed Approach for Handling Cold Start on New Items

The model implements **five complementary cold-start mechanisms for new items**:

### A. Item Cold-Start Validation Split

**Location**: `item_coldstart_split()` function

The function holds out entire movies (10-15%) to simulate real cold-start scenarios:
- Validation movies are completely unseen during training
- This tests how well the model handles new items using only content features
- Prevents data leakage by ensuring no validation item appears in training

### B. Heavy Bayesian Smoothing (m=80)

**Location**: `Config` and `build_target_encoding()`

Uses standard Bayesian smoothing with high strength (m=80):
```
smoothed = (n * avg + m * global_mean) / (n + m)
```

**Effect**: Items with few ratings are heavily pulled toward the global mean, preventing unreliable predictions for rare items.

### C. Content-Based Genre Features

**Location**: `prepare_features()` function

The wide component uses genre features (19 genres) extracted from movie metadata:
- Available for ALL items, even those with zero ratings
- Enables content-based recommendations for cold items
- Combined with year buckets for temporal information

### D. Zero-Initialized Item Bias

**Location**: `WideDeepModel._init_weights()`

Item biases start at zero:
- New items have neutral bias
- Predictions gracefully fall back to global mean + content features
- No extreme predictions for unseen items

### E. Global Mean Baseline

**Location**: `WideDeepModel.forward()`

Final prediction structure:
```
prediction = global_mean + user_bias + item_bias + wide_out + deep_out
```

Even with zero item bias and unlearned item embeddings, predictions are centered on a reasonable baseline using user embeddings and content features.

## 3. Assessment Methodology

### How We Assess Cold Start Effectiveness for New Items

To evaluate performance under true item cold-start conditions, we simulate the arrival of new movies by holding out entire items during training.

Procedure:
1. Randomly hold out 10% of movies from the dataset.
2. Train the recommender system only on the remaining movies.
3. Evaluate predictions only on ratings of the held-out movies, which are completely unseen during training.
4. Since these movies have no training ratings, the model must rely on: 
    - Content features (genres, release year) 
    - User representations
    - Global baseline statistics
This setup prevents information leakage and reflects a realistic new-item deployment scenario.

### Comparison Against Baselines

| Baseline | Description | Method |
|----------|-------------|--------|
| **Global Mean** | Predict average rating for all | `pred = global_mean` |
| **User Mean** | Average rating per user | `pred = mean(user_ratings)` |
| **Genre Mean** | Average rating per genre | `pred = mean(genre_ratings)` |

The Genre Mean baseline is a simple content-based method that serves as a strong non-personalized cold-item baseline.

### Evaluation Code Pattern

```python
# Split: hold out entire items (movies)
train_df, val_df = item_coldstart_split(data, val_item_ratio=0.10)

# Train model on non-cold items only
model = train_wide_deep(train_df)

# Baseline 1: Global mean
global_mean = train_df['rating'].mean()
baseline_global_rmse = rmse(val_df['rating'], global_mean)

# Baseline 2: User mean
user_means = train_df.groupby('user_id')['rating'].mean()
baseline_user_rmse = rmse(val_df['rating'], val_df['user_id'].map(user_means))

# Our model predictions on cold items
our_preds = predict_batch(model, val_df['user_id'], val_df['movie_id'])
our_rmse = rmse(val_df['rating'], our_preds)

improvement = (baseline_rmse - our_rmse) / baseline_rmse * 100
```

## 4. Implementation Details

### Key Implementation Sections:

#### 4.1 Configuration
- Heavy smoothing strength: 80.0
- User activity thresholds: (10, 40, 100)
- Item popularity thresholds: (30, 150, 400)

#### 4.2 Item Cold-Start Split Function
```python
def item_coldstart_split(df: pd.DataFrame, val_item_ratio: float = 0.10, random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split by ITEMS - hold out val_item_ratio% of movies entirely.
    This simulates cold-start scenario for new items not seen during training.
    """
    print(f"\n[ITEM COLD-START SPLIT] Holding out {val_item_ratio*100:.0f}% of items...")
    
    np.random.seed(random_state)
    
    # Get all unique items
    all_items = df['movie_id'].unique()
    n_items = len(all_items)
    n_val_items = int(n_items * val_item_ratio)
    
    # Shuffle and split items
    np.random.shuffle(all_items)
    val_items = set(all_items[:n_val_items])
    train_items = set(all_items[n_val_items:])
    
    # Split dataframe by items
    train_df = df[df['movie_id'].isin(train_items)].reset_index(drop=True)
    val_df = df[df['movie_id'].isin(val_items)].reset_index(drop=True)
    
    print(f"  Train items: {len(train_items):,}, Train ratings: {len(train_df):,}")
    print(f"  Val items (cold): {len(val_items):,}, Val ratings: {len(val_df):,}")
    print(f"  Avg ratings per train item: {len(train_df)/len(train_items):.1f}")
    print(f"  Avg ratings per val item: {len(val_df)/len(val_items):.1f}")
    
    return train_df, val_df
```

**Purpose**: Holds out 10% of movies entirely to simulate new items arriving in the system.

#### 4.3 Target Encoding with Leakage Prevention
```python
def build_target_encoding(self, train_df):
    # Computed ONLY on training data
    explicit = train_df[train_df['rating'].notna()]
    self.global_mean = explicit['rating'].mean()
    
    # Item stats with standard smoothing
    for mid, row in item_stats.iterrows():
        n, avg = row['count'], row['mean']
        smoothed = (n * avg + CONFIG.smoothing_strength * self.global_mean) / (n + CONFIG.smoothing_strength)
        self.item_mean_rating[mid] = smoothed
```

**Purpose**: Ensures item embeddings/biases are only learned from training items. Cold items get no training signal.

#### 4.4 Content Feature Extraction
Content features are extracted from movie metadata and are available for all items, including cold ones:
Genres: 19-dimensional multi-hot vector
Release year: Discretized into 6 buckets
```python
def prepare_features(user_ids, movie_ids):
    # Genre features (available for ALL movies)
    for i, mid in enumerate(movie_ids):
        if mid in FEATURES.genre_features:
            genre[i] = FEATURES.genre_features[mid]
    
    # Year bucket (one-hot, 6 buckets)
    year_bucket = FEATURES.movie_year_bucket.get(mid, 3)
    wide_features[i, offset + year_bucket] = 1.0
```

**Purpose**: Extracts content features (genres, year) that are available even for cold items with zero training ratings.

#### 4.5 Model Architecture
- **Item biases initialized to zero** for graceful cold-start handling
- **Wide part** uses content features (genres, year buckets)
- **Deep part** uses embeddings + content features
- **Final prediction** includes global mean baseline

```python
# Forward pass
prediction = global_mean + user_bias + item_bias + wide_out + deep_out
# For cold items: item_bias ≈ 0, item_emb is random
# Model relies on: user_bias, wide_out (content), user_emb
```

## 5. Demonstration of Effectiveness

### Actual Results

Based on item cold-start validation (movies unseen during training):

| Method | Cold Item RMSE | Notes |
|--------|----------------|-------|
| **Global Mean (Baseline)** | 1.0162 | No personalization or content |
| **User Mean (Baseline)** | 0.9199 | Uses user history only |
| **Our Model (Single)** | 0.8914 | Uses content features + user embeddings |

**Performance Improvements:**
- **12.3% improvement** over Global Mean baseline
- **3.1% improvement** over User Mean baseline

### Why the Model Handles Cold Items Well

1. **Content Features**: Genres (19-dimensional) and year information provide signal even for items with zero ratings.

2. **User Embeddings**: Known users have learned preferences that transfer to new items through content similarity.

3. **Zero-Initialized Item Bias**: New items start neutral, avoiding extreme predictions.

4. **Wide Tower**: Directly processes genre and year features to make content-based predictions.

5. **Hybrid Approach**: Combines collaborative filtering (user embeddings) with content-based features (genres, year).

### Cold Start Behavior Summary

| Scenario | User Bias | Item Bias | Wide (Content) | Deep | Prediction Quality |
|----------|-----------|-----------|----------------|------|-----------------|
| Both warm | Learned | Learned | Genre+Year | User+Item emb | Best |
| Cold item | Learned | ~0 | Genre+Year | User emb | Good (content+user) |
| Cold user | ~0 | Learned | Genre+Year | Item emb | Good (content+item) |
| Both cold | ~0 | ~0 | Genre+Year | ~0 | Reasonable (content only) |

**Key Insight**: For cold items, the model can leverage:
- User embeddings (learned preferences)
- Content features (genres, year)
- Wide tower (processes content directly)

This enables strong performance even on movies with no training ratings, achieving substantial improvements over naive baselines.

## Summary

The model handles **cold items (new movies)** through **five integrated mechanisms**:

| Mechanism | Purpose | Location |
|-----------|---------|----------|
| ✅ Item Cold-Start Split | Validate on unseen movies | `item_coldstart_split()` |
| ✅ Content Features (Genres+Year) | Signal for cold items | `prepare_features()` |
| ✅ User Embeddings | Known users + content → good predictions | `WideDeepModel` |
| ✅ Zero-Initialized Item Biases | Graceful degradation | `_init_weights()` |
| ✅ Global Mean Baseline | Safe default prediction | `forward()` |

**Key Advantage**: Even for movies with zero training ratings, the model can leverage:
- 19-dimensional genre vectors
- Year information (6 buckets + normalized)
- Learned user preferences
- Wide tower for direct content processing

This enables **content-based recommendations** that significantly outperform naive baselines, making the system robust to new item introductions.

In [18]:
## 6. Cold-Start Performance Demonstration (New Items)

print("\n" + "="*60)
print("COLD ITEM DEMONSTRATION")
print("="*60)

# Split data with item cold-start (hold out 10% of movies)
train_cold, val_cold = item_coldstart_split(train_explicit, val_item_ratio=0.10, random_state=42)

print(f"\nCold items: {val_cold['movie_id'].nunique()} movies never seen during training")
print(f"These items will test content-based features (genres, year)")

# Build features ONLY on training items
FEATURES_COLD = FeatureStore()
FEATURES_COLD.build_basic(train_cold, val_cold, movies_df)
FEATURES_COLD.build_target_encoding(train_cold)

# Train a single model on non-cold items
print("\n[Training model on non-cold items only]")
set_all_seeds(42)
original_features = FEATURES
FEATURES = FEATURES_COLD  # Temporarily swap to use cold features
model_cold, _, _ = train_model(train_cold, val_cold, seed=42)

# === BASELINE 1: Global Mean ===
baseline_global = train_cold['rating'].mean()
baseline_global_preds = np.full(len(val_cold), baseline_global)
baseline_global_rmse = np.sqrt(np.mean((val_cold['rating'] - baseline_global_preds) ** 2))

# === BASELINE 2: User Mean ===
user_means = train_cold.groupby('user_id')['rating'].mean().to_dict()
baseline_user_preds = val_cold['user_id'].map(lambda u: user_means.get(u, baseline_global)).values
baseline_user_rmse = np.sqrt(np.mean((val_cold['rating'] - baseline_user_preds) ** 2))

# === OUR MODEL ===
model_preds = predict_batch(model_cold, val_cold['user_id'].values, val_cold['movie_id'].values)
model_rmse = np.sqrt(np.mean((val_cold['rating'] - model_preds) ** 2))

# Restore original features
FEATURES = original_features

# === RESULTS ===
print(f"\n{'='*60}")
print("COLD ITEM RESULTS (Movies Never Seen During Training)")
print("="*60)
print(f"  Baseline (Global Mean):  RMSE = {baseline_global_rmse:.4f}")
print(f"  Baseline (User Mean):    RMSE = {baseline_user_rmse:.4f}")
print(f"  Our Model (Content):     RMSE = {model_rmse:.4f}")
print(f"\n  Improvement over Global: {(baseline_global_rmse - model_rmse)/baseline_global_rmse * 100:.1f}%")
print(f"  Improvement over User:   {(baseline_user_rmse - model_rmse)/baseline_user_rmse * 100:.1f}%")
print("="*60)

print("\n✓ Model successfully handles cold items using:")
print("  - Genre features (19 genres)")
print("  - Year information")
print("  - User embeddings (users are known)")
print("  - Global mean baseline")

# Clean up
del FEATURES_COLD, model_cold
gc.collect()
if DEVICE.type == 'mps': torch.mps.empty_cache()


COLD ITEM DEMONSTRATION

[ITEM COLD-START SPLIT] Holding out 10% of items...
  Train items: 1,800, Train ratings: 6,573,412
  Val items (cold): 200, Val ratings: 729,938
  Avg ratings per train item: 3651.9
  Avg ratings per val item: 3649.7

Cold items: 200 movies never seen during training
These items will test content-based features (genres, year)

BUILDING FEATURE STORE (Temporal Generalization)
[1/3] Building ID mappings...
  Users: 100,000, Items: 2,000
[2/3] Extracting movie metadata...
[3/3] Building genre features...
  Genres: 19

Building TARGET ENCODING (train data only)...
  Smoothing strength: 80.0
  Global mean: 3.6070
  Users with ratings: 100,000
  Items with ratings: 1,800

[Training model on non-cold items only]

TRAINING WIDE & DEEP (seed=42)
Config: emb=32, layers=(256, 128)
        dropout=0.35, weight_decay=0.005
        lr=0.0005, patience=6

Preparing features...
Train: 6,573,412, Val: 729,938
Model parameters: 3,422,738

Training...
  Epoch  1: Train=0.9541, V